<a href="https://colab.research.google.com/github/rrrudolph/forex/blob/main/SR_Zones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
!pip install finnhub-python
import finnhub
import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Zones


In [118]:
# import symbols

# Mock data
finnhub_higher_timeframes = {'M5': ['15', '30', '60'],
                            'M15': ['30', '60', 'D'],
                            'M30': ['60', 'D', 'W'],
                            'H1': ['60', 'D', 'W'],
                            }

htfs = finnhub_higher_timeframes['M5'] 

symbols = {'finnhub': ['MOO', 
                       'HYG', 
                       'VIXM', 
                       'VIXY', 
                       'XLF', 
                       'XLU', 
                       'XLY', 
                       'XLP', 
                       'IWF', 
                       'IWD', 
                       'BAC', 
                       'REET'],
           'mt5':  ''}

timeframes = ['15']

# This doesn't quite work the way it should because it needs
# to account for weekends. what a hassle.
seconds_per_candle = {'1': 60,
                      '5': 300,
                     '15': 900,
                     '30': 1800,
                     '60': 3600,
                      'D': 86400
                     }

fh_client = finnhub.Client(api_key="budmt1v48v6ped914310")

def finnhub_ohlc_request(symbol, timeframe, seconds=seconds_per_candle, num_candles=999):
    ''' Import note, this doesn't account for weekend gaps. So requesting small 
        amounts of M1 candles can cause problems on weekends.  Max returned candles
        seems to be about 650.  '''

    # Ensure timeframe is a string
    timeframe = str(timeframe)
    symbol = str(symbol)
    # Handle the start and end request times based on timeframe and number of candles requested
    end = round(time.time())
    start = end - (num_candles * seconds[timeframe])
    
    if 'OANDA' in symbol:
        r = requests.get(f'https://finnhub.io/api/v1/forex/candle?symbol={symbol}&resolution={timeframe}&from={start}&to={end}&token=budmt1v48v6ped914310')
    else:
        r = requests.get(f'https://finnhub.io/api/v1/stock/candle?symbol={symbol}&resolution={timeframe}&from={start}&to={end}&token=budmt1v48v6ped914310')
    
    # Check for a bad request
    r = r.json()
    if r['s'] == 'no_data':
        print('Finnhub OHLC Request error. Check parameters:')
        print(' - symbol:', symbol)
        print(' - timeframe:', timeframe)
        print(' - start:', start)
        print(' - end:  ', end)
        print('current time:', time.time())
    else:
        new_data = pd.DataFrame(data=r)

        # Format
        new_data = new_data.rename(columns={'t':'datetime', 'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume'})
        new_data = new_data[['datetime', 'open', 'high', 'low', 'close', 'volume']]
        # new_data.datetime = pd.to_datetime(new_data.datetime, unit='s')

        return new_data

def finnhub_sr_levels_request(fh_client, new_data, symbol, timeframe, htfs):
    # Get the SR levels
    levels = [fh_client.support_resistance(symbol, x) for x in htfs]
    if len(levels[0]) == 0:
        print(f'Failed to retrieve {symbol} {timeframe} SR levels.')
    else: 
        # Unpack each SR level into a col in new_data
        s = []
        for group in levels:
            for key, list_ in group.items():
                for i in list_:
                    s.append(i)
        new_data['sr_levels'] = s


In [119]:
# df = pd.DataFrame(data=mock_data)
df = finnhub_ohlc_request('OANDA:EUR_USD', 15)


In [120]:
df.head()

,datetime,open,high,low,close,volume
0,1609794900,1.22474,1.22476,1.22412,1.22414,353
1,1609795800,1.22414,1.22470,1.22411,1.22467,436
2,1609796700,1.22465,1.22518,1.22449,1.22509,414
3,1609797600,1.22509,1.22509,1.22437,1.22501,11
4,1609798500,1.22495,1.22500,1.22439,1.22486,33


In [121]:
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = pd.DataFrame()
    data['tr0'] = abs(df.high - df.low)
    data['tr1'] = abs(df.high - df.close.shift())
    data['tr2'] = abs(df.low - df.close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    df['atr'] = atr

atr(df)

In [122]:
# define peaks. can't use shift and rolling unfortunately.
df['peak_hi'] = df.high[
                (df.high > df.high.shift(-1))
                &
                (df.high > df.high.shift(-2))
                &
                (df.high > df.high.shift(-3))
                &
                (df.high > df.high.shift(-4))
                &
                (df.high > df.high.shift(-5))
                &
                (df.high > df.high.shift(-6))
                &
                (df.high > df.high.shift(-7))
                &
                (df.high > df.high.shift(-8))
                &
                (df.high > df.high.shift(-9))
                &
                (df.high > df.high.shift(-10))
                &
                (df.high >= df.high.shift(1))
                &
                (df.high >= df.high.shift(2))
                &
                (df.high >= df.high.shift(3))
                &
                (df.high >= df.high.shift(4))
                &
                (df.high >= df.high.shift(5))
                &
                (df.high >= df.high.shift(6))
                &
                (df.high >= df.high.shift(7))
                &
                (df.high >= df.high.shift(8))
                &
                (df.high >= df.high.shift(9))
                &
                (df.high >= df.high.shift(10))
]

df['peak_lo'] = df.low[
                 (df.low < df.low.shift(-1))
                 &
                 (df.low < df.low.shift(-2))
                 &
                 (df.low < df.low.shift(-3))
                 &
                 (df.low < df.low.shift(-4))
                 &
                 (df.low < df.low.shift(-5))
                 &
                 (df.low < df.low.shift(-6))
                 &
                 (df.low < df.low.shift(-7))
                 &
                 (df.low < df.low.shift(-8))
                 &
                 (df.low < df.low.shift(-9))
                 &
                 (df.low < df.low.shift(-10))
                 &
                 (df.low <= df.low.shift(1))
                 &
                 (df.low <= df.low.shift(2))
                 &
                 (df.low <= df.low.shift(3))
                 &
                 (df.low <= df.low.shift(4))
                 &
                 (df.low <= df.low.shift(5))
                 &
                 (df.low <= df.low.shift(6))
                 &
                 (df.low <= df.low.shift(7))
                 &
                 (df.low <= df.low.shift(8))
                 &
                 (df.low <= df.low.shift(9))
                 &
                 (df.low <= df.low.shift(10))
]

In [123]:
viz = df
fig = go.Figure(data=[go.Candlestick(x=viz.datetime,
                                        open=viz.open, 
                                        high=viz.high,
                                        low=viz.low, 
                                        close=viz.close)])

# Add peak high markers 
fig.add_trace(go.Scatter(
    x=df.datetime[df.peak_hi.notna()],
    y=df.peak_hi[df.peak_hi.notna()],
    mode="markers",
    name="Swing Ratings",
    # text=ratings,
    # yaxis= 'y2'
))


# Add peak low markers 
fig.add_trace(go.Scatter(
    x=df.datetime[df.peak_lo.notna()],
    y=df.peak_lo[df.peak_lo.notna()],
    mode="markers",
    name="Swing Ratings",
    # text=ratings,
    textposition="top center"
    # yaxis= 'y2'
))

fig.update_layout(
    autosize=False,
    width=1100,
    height=700,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4),
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
    )
fig.update(layout_xaxis_rangeslider_visible=False)

In [124]:

def set_peak_size(df):
    '''
    Get a rough estimate of how big each peak really is using a 10, 20, 40 step.
    '''

    ### Peak Highs 
    for i in df[df.peak_hi.notna()].index:
        # The 40 window
        max_forwards = df.loc[i+1:i+40, 'high'].max()
        max_backwards = df.loc[i-40:i, 'high'].max()

        if df.loc[i, 'peak_hi'] > max_forwards and df.loc[i, 'peak_hi'] >= max_backwards:
            df.loc[i, 'peak_size'] = 3
        
        else:
            # The 20 window
            max_forwards = df.loc[i+1:i+20, 'high'].max()
            max_backwards = df.loc[i-20:i, 'high'].max()

            if df.loc[i, 'peak_hi'] > max_forwards and df.loc[i, 'peak_hi'] >= max_backwards:
                df.loc[i, 'peak_size'] = 2

            else:
                df.loc[i, 'peak_size'] = 1



    ### Peak Lows 
    for i in df.peak_lo[df.peak_lo.notna()].index:
        # The 40 window
        min_forwards = df.loc[i+1:i+40, 'low'].min()
        min_backwards = df.loc[i-40:i, 'low'].min()

        if df.loc[i, 'peak_lo'] < min_forwards and df.loc[i, 'peak_lo'] <= min_backwards:
            df.loc[i, 'peak_size'] = 3
        
        else:
            # The 20 window
            min_forwards = df.loc[i+1:i+20, 'low'].min()
            min_backwards = df.loc[i-20:i, 'low'].min()

            if df.loc[i, 'peak_lo'] < min_forwards and df.loc[i, 'peak_lo'] <= min_backwards:
                df.loc[i, 'peak_size'] = 2

            else:
                df.loc[i, 'peak_size'] = 1
set_peak_size(df)

In [125]:
viz = df
fig = go.Figure(data=[go.Candlestick(x=viz.datetime,
                                        open=viz.open, 
                                        high=viz.high,
                                        low=viz.low, 
                                        close=viz.close)])

# Add peak high markers 
fig.add_trace(go.Scatter(
    x=df.datetime[df.peak_hi.notna()],
    y=df.peak_hi[df.peak_hi.notna()],
    mode="markers+text",
    name="",
    text=df.peak_size[df.peak_hi.notna()],
    textposition="top center"
    # yaxis= 'y2'
))

# Add peak low markers 
fig.add_trace(go.Scatter(
    x=df.datetime[df.peak_lo.notna()],
    y=df.peak_lo[df.peak_lo.notna()],
    mode="markers+text",
    name="",
    text=df.peak_size[df.peak_lo.notna()],
    textposition="bottom center"
    # yaxis= 'y2'
))


fig.update_layout(
    autosize=False,
    width=1100,
    height=700,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4),
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
    )
fig.update(layout_xaxis_rangeslider_visible=False)

In [126]:
def set_potential_zones(df):
    '''
    Based on each peaks size, give it a certain amount of time and
    ATR price range in which to find other peaks.
    '''
    
    # Create the time horizons (by bar count) based on peak sizes
    df['horizon'] = 50 * df.peak_size[df.peak_size.notna()] 

    # Create price range around each peak (the potential zones)
    # Note: this is going to be working off a pre-existing dataframe, otherwise
    #       I would just change the atr window value within the atr function itself
    for i in df[df.peak_hi.notna()].index:
        df.loc[i, 'upper'] = df.loc[i, 'peak_hi'] + df.atr.rolling(40).mean()[i] * df.loc[i, 'peak_size'] / 2 
        df.loc[i, 'lower'] = df.loc[i, 'peak_hi'] - df.atr.rolling(40).mean()[i] * df.loc[i, 'peak_size'] / 2

    for i in df[df.peak_lo.notna()].index:
        df.loc[i, 'upper'] = df.loc[i, 'peak_lo'] + df.atr.rolling(40).mean()[i] * df.loc[i, 'peak_size'] / 2
        df.loc[i, 'lower'] = df.loc[i, 'peak_lo'] - df.atr.rolling(40).mean()[i] * df.loc[i, 'peak_size'] / 2
        
set_potential_zones(df)

In [127]:
def confirm_zones(df):
    '''
    Use the potential zone parameters to check for other peaks
    that fall within those time and price boundaries.
    '''

    # The process can be hi/lo agnostic at this point
    for i in df[df.upper.notna()].index:
        zone = df[
                  (df.index > i - df.loc[i, 'horizon'] / 4)  # set a slight look back window
                  &
                  (df.index < i + df.loc[i, 'horizon'])
                  &
                  (((df.peak_lo < df.loc[i, 'upper']) &
                  (df.peak_lo > df.loc[i, 'lower']))
                  |
                  ((df.peak_hi < df.loc[i, 'upper']) &
                  (df.peak_hi > df.loc[i, 'lower'])))           
        ]

        # Note: now that Im setting a look back window I will start double counting zones
        # However, rather than having smaller peaks take on the zone size of the largest peak
        # found in a zone, maybe it will be easier now to loop through the zones and delete 
        # the zones of smaller peaks that exist in larger peak's zones.  Because even if
        # the zone_start gets set to the first peak, the data still exists on the row of
        # the larger peak. damn im tired. wtf does all this mean.

        # Set the zone parameters on the current row
        # (this isn't working based off the largest peak found,
        # its only looking at the first peaks peak_size)
        if len(zone) > 1:
            df.loc[i, 'zone_start'] = zone.index.min()

            # Set the end based off the largest peak_size found in the zone
            end = zone.index[zone.peak_size == zone.peak_size.max()]
            df.loc[i, 'zone_end'] = end[-1] + df.loc[i, 'horizon']
            
            # For the upper and lower values I'll just re-use what already exists

confirm_zones(df)

In [128]:
print(len(df[df.peak_size.notna()]))
print(len(df[df.zone_start.notna()]))

53
31


In [129]:
t = [1,1]
max(t)

1

In [130]:
# Create some objects for plotting
plot_x = []
plot_y = []
df.zone_end[df.zone_end > df.index.max()] = df.index.max()
for i in df[df.zone_start.notna()].index:
    start = df.loc[i, 'zone_start']
    end = df.loc[i, 'zone_end']
    
    plot_x.append(df.loc[start, 'datetime'])
    plot_x.append(df.loc[end, 'datetime'])
    plot_x.append(df.loc[end, 'datetime'])
    plot_x.append(df.loc[start, 'datetime'])
    plot_x.append(df.loc[start, 'datetime'])
    plot_x.append(None)
    # full zone
    plot_y.append(df.loc[i, 'lower'])
    plot_y.append(df.loc[i, 'lower'])
    plot_y.append(df.loc[i, 'upper'])
    plot_y.append(df.loc[i, 'upper'])
    plot_y.append(df.loc[i, 'lower'])
    plot_y.append(None)

# Plot them
viz = df
fig = go.Figure(data=[go.Candlestick(x=viz.datetime,
                                        open=viz.open, 
                                        high=viz.high,
                                        low=viz.low, 
                                        close=viz.close)])

# Add peak high markers 
fig.add_trace(go.Scatter(
    x=df.datetime[df.peak_hi.notna()],
    y=df.peak_hi[df.peak_hi.notna()],
    mode="markers+text",
    name="",
    text=df.peak_size[df.peak_hi.notna()],
    textposition="top center"
    # yaxis= 'y2'
))

# Add peak low markers 
fig.add_trace(go.Scatter(
    x=df.datetime[df.peak_lo.notna()],
    y=df.peak_lo[df.peak_lo.notna()],
    mode="markers+text",
    name="",
    text=df.peak_size[df.peak_lo.notna()],
    textposition="bottom center"
    # yaxis= 'y2'
))

# Zones
fig.add_trace(go.Scatter(x=plot_x, y=plot_y, fill="toself"))


fig.update_layout(
    autosize=False,
    width=1100,
    height=700,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4),
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
    )
fig.update(layout_xaxis_rangeslider_visible=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [131]:
def delete_expired_zones(df):
    '''
    Based on the last peak which makes up a zone, count how many times
    price crosses through the zone.  Once price crosses through twice the
    zone will be deleted, however if a new peak gets created in the zone
    the count will naturally reset.
    '''